
## Overview

Este notebook tem a função de transformar o arquivo de servidores públicos, extraída do Portal da Transparência do Governo Federal, em tabelas mais normalizadas, as quais possam ser armazenadas no formato PySpark e utilizadas quando necessário.

In [0]:
# File location and type
file_location = "/FileStore/tables/202401_Cadastro.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ";"
encode = 'latin1'

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .option("encoding", encode) \
  .load(file_location)

# display(df)

In [0]:
# Separar tabelas de dados para evitar duplicações desnecessárias
df_cad_servidores = df[["id_SERVIDOR_PORTAL", "NOME", "CPF"]].drop_duplicates(subset=['id_SERVIDOR_PORTAL'])

Cria tabela permanente de cadastro de servidores

In [0]:
# criar tabela de servidores
permanent_table_name = "cadastro_servidores"
df_cad_servidores.write.mode("overwrite").saveAsTable(permanent_table_name)

# df.write.format("parquet").saveAsTable(nome_da_tabela)

In [0]:
# separar unidades organizacionais
df_uorgs = df[["COD_UORG_LOTACAO", "UORG_LOTACAO"]].drop_duplicates()

In [0]:
# criar tabela de unidades organizacionais
permanent_table_name = "unid_org"
# df_uorgs.write.format("parquet").saveAsTable(permanent_table_name)
df_uorgs.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
# retirar descrição da UORG do df original
df = df.drop("UORG_LOTACAO")

In [0]:
# separar organização lotação
df_org_lot = df[["COD_ORG_LOTACAO", "ORG_LOTACAO"]].drop_duplicates()

In [0]:
# criar tabela de unidades organizacionais
permanent_table_name = "org_lotacao"
# df_org_lot.write.format("parquet").saveAsTable(permanent_table_name)
df_org_lot.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
# retirando a descrição do órgão lotação do dataframe principal
df = df.drop("ORG_LOTACAO")

In [0]:
df_org_sup = df[["COD_ORGSUP_LOTACAO", "ORGSUP_LOTACAO"]].drop_duplicates()

In [0]:
permanent_table_name = "org_lotacao_superior"
# df_org_sup.write.format("parquet").saveAsTable(permanent_table_name)
df_org_sup.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
df = df.drop("ORGSUP_LOTACAO")

In [0]:
cols_to_drop = ['NOME', 'CPF', 'MATRICULA', 'CLASSE_CARGO', 'REFERENCIA_CARGO', 'PADRAO_CARGO', 
               'NIVEL_CARGO', 'SIGLA_FUNCAO', 'NIVEL_FUNCAO', 'CODIGO_ATIVIDADE', 'OPCAO_PARCIAL', 
               'COD_UORG_EXERCICIO', 'UORG_EXERCICIO', 'COD_ORG_EXERCICIO', 'ORG_EXERCICIO', 'COD_ORGSUP_EXERCICIO', 
               'ORGSUP_EXERCICIO', 'COD_TIPO_VINCULO', 'DATA_INICIO_AFASTAMENTO', 'DATA_TERMINO_AFASTAMENTO',
               'REGIME_JURIDICO', 'JORNADA_DE_TRABALHO', 'DATA_INGRESSO_CARGOFUNCAO', 'DATA_NOMEACAO_CARGOFUNCAO', 
               'DOCUMENTO_INGRESSO_SERVICOPUBLICO', 'DATA_DIPLOMA_INGRESSO_SERVICOPUBLICO', 'DIPLOMA_INGRESSO_CARGOFUNCAO',
               'DIPLOMA_INGRESSO_ORGAO', 'DIPLOMA_INGRESSO_SERVICOPUBLICO', 'UF_EXERCICIO'
               'DIPLOMA_INGRESSO_CARGOFUNCAO', 'UF_EXERCICIO'
               ]

In [0]:

# retirar diversas colunas que não são necessárias para respondera as questões
df = df.drop(*cols_to_drop)

In [0]:
permanent_table_name = "qualificacao_servidores"
# df.write.format("parquet").saveAsTable(permanent_table_name)
df.write.mode("overwrite").saveAsTable(permanent_table_name)

In [0]:
%sql
SELECT a.NOME, b.FUNCAO, c.ORG_LOTACAO  
FROM cadastro_servidores a 
JOIN qualificacao_servidores b ON a.Id_SERVIDOR_PORTAL = b.Id_SERVIDOR_PORTAL 
JOIN org_lotacao c ON b.COD_ORG_LOTACAO = c.COD_ORG_LOTACAO
WHERE b.DESCRICAO_CARGO = 'TECNICO DO SEGURO SOCIAL' AND c.ORG_LOTACAO <> 'Instituto Nacional do Seguro Social' LIMIT 10

NOME,FUNCAO,ORG_LOTACAO
WILMA BOECHAT DE SOUZA SOARES,Sem informação,MINISTERIO DA FAZENDA
MARGARETH ZIMMERMANN,Sem informação,MINISTERIO DA FAZENDA
SOLANGE APARECIDA DO NASCIMENTO SOARES,Sem informação,MINISTERIO DA FAZENDA
CAROLINA BEATRIZ IMTHURN,Sem informação,MINISTERIO DA FAZENDA
PAULO ROBERTO BOTIN,Sem informação,MINISTERIO DA FAZENDA
BRASILIANO DIAS DOS SANTOS,Sem informação,MINISTERIO DA FAZENDA
MARIA LUCIANA PEREIRA DYSMAN SEIXAS,Sem informação,MINISTERIO DA FAZENDA
ROBERTO CARLOS DE ARAUJO,Sem informação,MINISTERIO DA FAZENDA
RICARDO VIANNA JORGE,Sem informação,MINISTERIO DA FAZENDA
VALERIA SILVA FERREIRA DA COSTA,Sem informação,MINISTERIO DA FAZENDA
